In [1]:
import pandas as pd
import math
import random
import numpy as np

In [2]:
iris=pd.read_csv('/Users/glitterlungs/Studia/SEMESTR IV/Systemy sztucznej inteligencji/Zadanie nr 1/iris.csv')
print(iris)

     sepal.length  sepal.width  petal.length  petal.width    variety
0             5.1          3.5           1.4          0.2     Setosa
1             4.9          3.0           1.4          0.2     Setosa
2             4.7          3.2           1.3          0.2     Setosa
3             4.6          3.1           1.5          0.2     Setosa
4             5.0          3.6           1.4          0.2     Setosa
..            ...          ...           ...          ...        ...
145           6.7          3.0           5.2          2.3  Virginica
146           6.3          2.5           5.0          1.9  Virginica
147           6.5          3.0           5.2          2.0  Virginica
148           6.2          3.4           5.4          2.3  Virginica
149           5.9          3.0           5.1          1.8  Virginica

[150 rows x 5 columns]


In [3]:
class ProcessingData:
    @staticmethod
    def shuffleDF(data:pd.DataFrame)->pd.DataFrame:
        for i in range(len(data)-1,0,-1):
            x=random.randint(0,i)
            data.iloc[i], data.iloc[x] = data.iloc[x], data.iloc[i]
        return data
    
    @staticmethod
    def normalizeDF(data:pd.DataFrame)->pd.DataFrame:
        col=data.columns.tolist()
        col=col[:-1]
        for j in col:
            minn=data[j].min()
            maxx=data[j].max()
            for i in range(0,len(data),1):
                data.at[i,j]=(float(data.at[i,j])- minn)/(maxx - minn)
        return data
    
    @staticmethod
    def splitDF(data:pd.DataFrame)->pd.DataFrame:
        trainDF=pd.DataFrame(columns=data.columns)
        testDF=pd.DataFrame(columns=data.columns)
        for i in range(len(data)):
            if i <= len(data)*0.7:
                trainDF=trainDF.append(data.iloc[i])
            else:
                testDF=testDF.append(data.iloc[i])
        return trainDF, testDF

In [31]:
class NaiveBayesClassifier:
    # srednia arytmetyczna
    @staticmethod
    def mean(atr: list):
        return sum(atr)/len(atr)
    
    # odchylenie standardowe
    @staticmethod
    def std(atr: list):
        # atr to lista
        return np.std(atr)
    
    # gauss
    @staticmethod
    def gauss(sampleAtr, mean, std):
        exponent = np.exp(-(sampleAtr-mean)**2/(2*std**2))
        return 1/(np.sqrt(2*np.pi*std))*exponent
    
    # klasyfikator Bayesa
    @staticmethod
    def classify(df: pd.DataFrame, sample: list):
        varieties = pd.unique(df["variety"])
        attributes = df.columns.tolist()
        attributes = attributes[:-1]
        
        #seperacja df na klasy - dict z variety, w kazdym z variety dict z atributes i w kazdym atributes lista wartosci
        variety_atr = {} 
        mean_std_arr = np.zeros((3,4,2))
        for var in varieties:
            variety_atr[var] = {}
            for atr in attributes:
                variety_atr[var][atr] = []
                for i in range (len(df)):
                    if(df.at[i, "variety"] == var):
                        variety_atr[var][atr].append(df.at[i,atr])
                        # stworzenie tablicy trzywymiarowej [[[m,s]*4]*3], dla kazdego gatunku 4 tablice reprezentujace atrybuty, a w nich dla kazego atrybutu obliczone mean i std
                mean_std_arr[np.where(varieties == var)[0][0]][attributes.index(atr)][0] = NaiveBayesClassifier.mean(variety_atr[var][atr])
                mean_std_arr[np.where(varieties == var)[0][0]][attributes.index(atr)][1] = NaiveBayesClassifier.std(variety_atr[var][atr])
        
        # obliczenie składowych prawdopodobieństw (gauss) dla każdego gatunku zapis w dict
        result = {}
        for var in range(len(varieties)):
            result[varieties[var]] = 1/len(varieties)
            for atr in range(len(attributes)):
                result[varieties[var]] *= NaiveBayesClassifier.gauss(sample[atr], mean_std_arr[var][atr][0],mean_std_arr[var][atr][1])
        
        return max(result, key=result.get)
    
    @staticmethod      
    def precision(testData: pd.DataFrame, trainData: pd.DataFrame):
        good = 0
        bad = 0
        for sample in testData.values:
            if NaiveBayesClassifier.classify(trainData, sample) == sample[-1]:
                good += 1
            else:
                bad += 1
                print("Sample " + sample[-1] + " classifiesd as: " + NaiveBayesClassifier.classify(trainData, sample))
        return good/(bad+good)*100
        
        
        
    # w petli dla wszystkich próbek w bazie walidacyjnej
    
    
    # obliczamy dokładność klasyfikatora

In [34]:
iris=ProcessingData.shuffleDF(iris)
iris=ProcessingData.normalizeDF(iris)
train,test=ProcessingData.splitDF(iris)
print(train)
print(test)
print(iris)

print(f"len(train_X):{len(train)}")
print(f"len(test_X):{len(test)}")
print("--------------")

result = NaiveBayesClassifier()
t1 = result.mean([4,5,6])
t2 = result.std([4,5,6])
t3 = result.gauss(4,t1,t2)
t5 = result.precision(test, train)
print(t5)

     sepal.length  sepal.width  petal.length  petal.width     variety
0        0.638889     0.416667      0.576271     0.541667  Versicolor
1        0.388889     0.333333      0.525424     0.500000  Versicolor
2        0.611111     0.416667      0.813559     0.875000   Virginica
3        0.027778     0.375000      0.067797     0.041667      Setosa
4        0.194444     0.541667      0.067797     0.041667      Setosa
..            ...          ...           ...          ...         ...
101      0.111111     0.500000      0.050847     0.041667      Setosa
102      0.361111     0.416667      0.525424     0.500000  Versicolor
103      0.166667     0.208333      0.593220     0.666667   Virginica
104      0.861111     0.333333      0.864407     0.750000   Virginica
105      0.722222     0.458333      0.694915     0.916667   Virginica

[106 rows x 5 columns]
     sepal.length  sepal.width  petal.length  petal.width     variety
106      0.583333     0.333333      0.779661     0.875000   Virgin